In [5]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'python', 'make_dataset.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'python', 'make_dataset.ipynb'], returncode=0)

In [1]:
# モジュールインポート
#-*- coding:utf-8 -*-

import os
import json
import numpy as np
from collections import namedtuple, Counter
import tensorflow as tf

tf.flags.DEFINE_string('train_img_dir', 'data/img/train2014/', 'Training image directory.')
tf.flags.DEFINE_string('val_img_dir', 'data/img/val2014/', 'validation image directory.')
tf.flags.DEFINE_string('train_captions', 'data/stair_captions_v1.2_train.json', 'Training caption file.')
tf.flags.DEFINE_string('val_captions', 'data/stair_captions_v1.2_val.json', 'Validation caption file.')
tf.flags.DEFINE_string('out_dir', 'data/tfrecords/', 'Output TFRecords directory.')
tf.flags.DEFINE_integer('min_word_count', 4, 'The minimum number of occurrences of each word in the training set for includion in the vocab.')
tf.flags.DEFINE_string('word_list_file', 'data/dictinary.txt', 'Output word list file.')

FLAGS = tf.flags.FLAGS

START_WORD = '<S>'
END_WORD = '<E>'
UNKNOWN_WORD = '<UNW>'

NUM_TRAIN_FILE = 256
NUM_VAL_FILE = 4
NUM_TEST_FILE = 8

ImageMetadata = namedtuple('ImageMetadata', ['img_id', 'filename'])
print(ImageMetadata)

<class '__main__.ImageMetadata'>


In [2]:
#Jsonファイルを読み込み画像のid、ファイル名、キャプションを取得する。
def _load_metadata(caption_filename, img_dir):

    #jsonファイルをロード
    with open(caption_filename, 'r') as f:
        meta_data = json.load(f)

    #画像idとファイル名を持つnamedtupleのリストを作成
    meta_list = [ImageMetadata(x['id'], os.path.join(img_dir, x['file_name'])) for x in meta_data['images']]

    #スペース区切りのcaptinを単語の配列に変換
    def _create_word_list(caption):
        tokenized_captions = [START_WORD]
        tokenized_captions.extend(captin.split())
        tokenized_captions.append(END_WORD)
        return tokenized_captions

    #{画像id => キャプションのリスト}の辞書を作成
    id_to_captions = {}
    for annotatin in meta_data['annotations']:
        img_id = annotation['image_id']
        caption = annotation['tokenized_caption']
        caption = _create_word_list(caption)
        #キャプションはいくつかあるため１つだけ採用
        id_to_captions[img_id] = caption

    print('Loaded caption metadata for %d images from %s' %
            len(meta_list), caption_filename)

    return meta_list, id_to_captions

In [3]:
def main(argv):

    #jsonファイルからメタデータの読み込み
    #(画像id、ファイルパス)のタプるの配列{id=>キャプションのリスト}を取得
    train_meta, train_captions = _load_metadata(FLAGS.train_captions, FLAGS.train_img_dir)
    val_meta, val_captions = _load_metadata(FLAGS.val_captions, FLAGS.val_img_dir)

    #キャプションをマージ
    captions = {k:v for dic in [train_captions, val_captions]
            for k, v in dic.items()}